<a href="https://colab.research.google.com/github/naguzmans/opportunistic-atm/blob/main/opensky_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install PyOpenSky

In [ ]:
!pip install pyopensky --upgrade
!pip install haversine
!mv secret.conf /root/.config/pyopensky/

Perform Query

In [11]:
import pandas as pd
import datetime
from haversine import haversine, inverse_haversine, Unit
import numpy as np
from pyopensky import OpenskyImpalaWrapper

# Time of query
def to_epoch(timestamp):
  unix_epoch = datetime.datetime(1970, 1, 1)
  log_dt = datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
  seconds_from_epoch = (log_dt - unix_epoch).total_seconds()
  return seconds_from_epoch

from_time_str = '2022-04-22 13:00:00'
to_time_str = '2022-04-22 14:00:00'
from_time = to_epoch(from_time_str)
to_time = to_epoch(to_time_str)

# Area of query
airport = (51.871496514, -0.367665196) # Luton
diag_distance = 10
min_coord = inverse_haversine(airport, diag_distance, np.pi * 1.25)
max_coord = inverse_haversine(airport, diag_distance, np.pi * .25)
min_lat, max_lat, min_lon, max_lon = (min_coord[0], max_coord[0], min_coord[1], max_coord[1])

# Height of query
min_height, max_height = (0, 2000)

# OpenSky wrapper
opensky = OpenskyImpalaWrapper()
os_query = opensky.rawquery(
    f'SELECT * FROM state_vectors_data4 \
    WHERE lat >= {min_lat} AND lat <= {max_lat} \
    AND lon >= {min_lon} AND lon <= {max_lon} \
    AND hour >= {from_time} AND hour <= {to_time} \
    AND geoaltitude >= {min_height} AND geoaltitude <= {max_height};'
)

unique_callsigns = pd.unique(os_query['callsign'])
print(f'Query length: {len(unique_callsigns)}')

* Server connection successful!
* Fetching records...
Sending query request: [SELECT * FROM state_vectors_data4     WHERE lat >= 51.80786007740522 AND lat <= 51.93504302967222     AND lon >= -0.47051413811321224 AND lon <= -0.2645249782580956     AND hour >= 1650632400.0 AND hour <= 1650636000.0     AND geoaltitude >= 0 AND geoaltitude <= 2000;]
Processing query result.
* Records downloaded.
Query length: 38


In [14]:
os_query.head()

,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact,hour
2281,1650632401,4072e5,51.829971,-0.377766,45.866368,336.885792,-3.25120,GOLEA,False,False,False,4674.0,548.64,525.78,1.650632e+09,1.650632e+09,1650632400
880,1650632402,400d8c,51.874649,-0.368116,66.778715,74.357754,0.65024,EZY82MV,False,False,False,NaN,213.36,228.60,1.650632e+09,1.650632e+09,1650632400
2282,1650632402,4072e5,51.831122,-0.378494,45.866368,336.885792,-2.60096,GOLEA,False,False,False,4674.0,548.64,525.78,1.650632e+09,1.650632e+09,1650632400
2283,1650632403,4072e5,51.831482,-0.378806,46.618960,337.963773,-0.32512,GOLEA,False,False,False,4674.0,548.64,525.78,1.650632e+09,1.650632e+09,1650632400
881,1650632403,400d8c,51.874649,-0.368116,66.778715,74.357754,0.65024,EZY82MV,False,False,False,NaN,213.36,228.60,1.650632e+09,1.650632e+09,1650632400


In [15]:
len(os_query)

4215